
<center>
    <h1> INF285 - Computación Científica </h1>
    <h1> Tarea N°1, V0.3 </h1>
    
</center>

<p>

</p>

## Instrucciones

* La tarea es individual, sin embargo se invita a todos l@s estudiantes a que discutan entre ustedes las preguntas pero luego implementen de forma individual su tarea.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
* **No modifique la firma de las funciones** (a menos que se le diga lo contrario) y respete el output que se le exije. **En caso de no respetar esta regla la función se considerará errónea.**
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 6 de mayo a las **18:00 hrs**.  
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).
* Puede agregar funciones extras siempre y cuando **no interfieran en las firmas** de las funciones principales establecidas.

## Introducción 

En esta primera tarea de INF-285, versión 2021-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Importancia, Errores de Cancelación y Búsqueda de Ceros. 
El desarrollo de cada uno de esos temas se presenta en una serie de preguntas donde deberá ir decidiendo, pregunta a pregunta, cómo cada uno de los temas se aplica.
En general, los temas no se analizan por separado, sino de manera acoplada. 
Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. 
Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.
Es decir, debe aplicar la metodología IDEA!

## Problemas

### 1. Simulador (60 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=-1 + x + 2 A^2 x - 3 A^2 x^2 + A^2 x^3
\end{equation}

Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$), 'bits_mant' que corresponde a la cantidad de bits que tiene la mantisa y 'bits_exp' que corresponde a la cantidad de bits que tiene el exponente. 
El algoritmo se ejecutará en un computador ficticio que representa los números con esa cantidad de bits para el estándar de punto flotante IEEE. 
Debe controlar los casos especiales correctamente.
Esta implementación debe calcular las raíces de $f$ sin pérdida de **importancia** con la cantidad de bits disponibles para la mantisa y la cantidad de bits para el exponente.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

El computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces. 
Considere como límites de bits para la mantisa en el intervalo $[2,52]$ y en el intervalo $[2,11]$ para los bits del exponente.

Ejemplo:

```python
# Algoritmo de ejemplo
a = 9819824.624837
b = 148736.523476
c = a+b

# Implementación de ejemplo con Representación de Punto Flotante de 'bits_mant' bits en la mantisa y 'bits_exp' bits en el exponente.
am = f_new_rep(9819824.624837,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
cm = f_suma(am,bm,bits_mant,bits_exp) # Aproximar el output de la suma y cada operación en la nueva representación.
```

## Explicación:

Utilizando la función anterior:

\begin{equation}
f(x)=-1 + x + 2 A^2 x - 3 A^2 x^2 + A^2 x^3
\end{equation}

Podemos ver que existe una raíz en x=1, por lo que si dividimos la función utilizando una división sintética por (x-1) obtenemos:

\begin{equation}
f(x)=-1 + x + 2 A^2 x - 3 A^2 x^2 + A^2 x^3 = (A^2 x^2 - 2 A^2x + 1)(x - 1)
\end{equation}

Ahora solo se debe resolver la ecuación cuadrática a través de la fórmula de ecuación de segundo grado:

\begin{equation}
x = \frac{-b ± \sqrt{ b^2 - 4ac}}{2a}
\end{equation}

Siendo:
\begin{equation}
a=A^2, b=-2A^2, c=1
\end{equation}

Debido a que b < 0 debemos evitar el caso donde exista

\begin{equation}
-b - \sqrt{ b^2 - 4ac}
\end{equation}

Esto es porque podría existir una pérdida de importancia cuando se ejecuta una resta.
Por lo que racionalizando obtenemos las 3 raíces:

\begin{equation}
x_1 = \frac{-b + \sqrt{ b^2 - 4ac}}{2a}
\end{equation}

<br>

\begin{equation}
x_2 = \frac{-b - \sqrt{ b^2 - 4ac}}{2a} \cdot \frac{-b + \sqrt{ b^2 - 4ac}}{-b + \sqrt{ b^2 - 4ac}} = \frac{2c}{-b + \sqrt{ b^2 - 4ac}}
\end{equation}

<br>

Reemplazando obtenemos:


\begin{equation}
x_1 = \frac{2A^2 + \sqrt{ 4A^4 - 4A^2}}{2A^2} = \frac{2A^2 + 2A\sqrt{ A^2 - 1}}{2A^2} = 1 + \frac{\sqrt{ A^2 - 1}}{A}
\end{equation}

<br>

\begin{equation}
x_2 = \frac{2}{2A^2 + \sqrt{ 4A^4 - 4A^2}} = \frac{1}{(A^2 + A\sqrt{ A^2 - 1})}
\end{equation}

<br>

\begin{equation}
x_3 = 1
\end{equation}

<br>

In [22]:
import bitstring as bs
import numpy as np

def isNaN(num):
    return num != num


#suma un 1 a la mantisa de un numero binario
def sumarUno(mantisa,bits_mant):
    mantisa = mantisa[:bits_mant]
    aux = mantisa[:bits_mant]
    for i in range(bits_mant-1,-1,-1):
        if(mantisa[i] == "0"):
            mantisa = mantisa[:i] + "1" + mantisa[i+1:]
            break
        if(mantisa[i] == "1"):
            mantisa = mantisa[:i] + "0" + mantisa[i+1:]
            if(i == 0):
                return aux
    return mantisa

#redondea o trunca correspondientemente
def redondear(mantisa,bits_mant):
    if(mantisa[bits_mant] == "0"): #53
        print("truncar 1")
        return mantisa[:bits_mant]
    if(mantisa[bits_mant] == "1"): #53
        if(len(mantisa) > bits_mant): #54
            for i in range(bits_mant+1,len(mantisa)):
                if(mantisa[i] == "1"):#54,55,56...
                    print("redondear 1")
                    return sumarUno(mantisa,bits_mant)
        if(mantisa[bits_mant-1] == "1"):#52
            print("redondear 2")
            return sumarUno(mantisa,bits_mant)
        if(mantisa[bits_mant-1] == "0"):#52
            print("truncar 2")
            return mantisa[:bits_mant]

#transforma en binario la parte entera de un número de base 10
def parteEntero(num):
    if("-" in num):
        return bin(int(num))[3:]
    else:
        return bin(int(num))[2:]

#transforma en binario la parte decimal de un número de base 10
def parteDecimal(num):
    if(num == 0):
        return "0"
    num = float("0."+str(num))
    resultado = ""
    while (num != 0):
        parteDec = str(int(num*2))
        resultado += str(int(num*2))
        num = float(num*2) - float(parteDec)
    return resultado
    

#transforma un número base 10 a binario
def convertirBinario(num):
    num = float(num)
    #if ("e" in str(num)):
    num = f"{num:.52f}"
    
    if(num == 0 or num == 0.0):
        return ("0","0","0")
    num = str(num)
    entero = ""
    decimal = ""
    signo = "0"
    if ("-" in num):
        signo = "1"
    if ("." in num):
        entero,decimal = num.split(".")
        if("-" in entero):
            entero = entero[1:]
        entero = parteEntero(entero)
        decimal = parteDecimal(decimal)
    else: #cuando es solo un numero entero
        entero = parteEntero(num)
    return (signo,entero,decimal)
    
#devuelve el rango de exponente posibles con bits_exp
def rangoExponente(bits_exp):
    cotaSup = (2**(bits_exp-1)-1)
    cotaInf = -(cotaSup - 1)
    return range(cotaInf,cotaSup+1)

#mueve el punto del número binario para transformarlo en ieee, calzando el exponente
def calzarExponente(signo,entero,decimal,bits_exp): #devuelve el numero como 0 00000 000000000
    rangoExp = rangoExponente(bits_exp)
    if(entero != "0"):
        if ((len(entero)-1) in rangoExp):
            #print("exponente positivo correcto")
            exp = str(len(entero)-1 + rangoExp[-1])
            _,exp,_ = convertirBinario(exp)
            return signo + " " + exp + " " + entero[1:]+decimal
        else: #si esta fuera del rango
            #print("exponente positivo incorrecto")
            entero = entero[:rangoExp[-1]+1]
            exp = str(len(entero)-1 + rangoExp[-1])
            _,exp,_ = convertirBinario(exp)
            exp = exp[:-1] + "1"
            return signo + " " + exp + " " + entero[1:]+decimal
    else:
        primerUno = 0
        for i in range(len(decimal)):
            if(decimal[i] == "1"):
                primerUno = i+1
                break
        if (primerUno == 0):
            return "0 0 0"
        if ((-1*primerUno) in rangoExp):
            #print("exponente negativo correcto")
            exp = str(-1*primerUno + rangoExp[-1])
            _,exp,_ = convertirBinario(exp)
            while(len(exp)<bits_exp):
                exp = "0" + exp
            return signo + " " + exp + " " + decimal[primerUno:]
        else: #cuando no alcanza el exp negativo
            #print("exponente negativo incorrecto")
            return "0 0 0"
            
            
#revisa si la mantisa del numero en ieee se necesita redondear o no
def calzarMantisa(numPF,bits_mant):
    signo,exp,mantisa = numPF.split(" ")
    if (bits_mant >= len(mantisa)):
        #print("mantisa correcta")
        return numPF
    else:
        #print("mantisa incorrecta")
        mantisa = redondear(mantisa,bits_mant)
        return signo+" "+exp+" "+mantisa
    
#transforma un numero binario entero a base 10
def binarioEntero(binario):
    valor = 0
    for  i in range(len(binario)-1,-1,-1):
        if(binario[i] == "1"):
            valor += 2**(len(binario)-1-i)
    return valor

#transforma un numero binario decimal a base 10
def binarioDecimal(binario):
    valor = 0
    for  i in range(len(binario)):
        if(binario[i] == "1"):
            valor += 2**(-(i+1))
    return valor
        
#transforma un número ieee a base 10
def convertirDecimal(numPF,bits_mant,bits_exp,entero):
    signo,exp,mantisa = numPF.split(" ")
    if (exp == "0" and mantisa == "0"):
        return "0"
    if ("0" not in exp and len(exp) == bits_exp):
        if (signo == "0"):
            return np.inf
        if (signo == "1"):
            return np.NINF
    numero = ""
    if(signo == "1"):
        numero = numero + "-"
    exp = binarioEntero(exp)
    exp = exp - (2**(bits_exp-1)-1)
    if(entero == "0"):
        decimal = "1" + mantisa
        for i in range(-exp-1):
            decimal = "0" + decimal
    else:
        entero = "1" + mantisa[:exp]
        decimal = mantisa[exp:]
    if(exp>(len(entero)-1)):
        for i in range(exp-(len(entero)-1)):
            entero = entero + "0"
    print("e",entero)
    print("d",decimal)
    entero = binarioEntero(entero)
    decimal = binarioDecimal(decimal)
    if(decimal == 0):
        numero = numero + str(entero) + ".0"
    else:
        numero = numero + str(entero+decimal)
    return numero
    

"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_new_rep(x,bits_mant,bits_exp):
    # Algoritmo de representación de punto flotante modificada.
    if(isNaN(x)):
        return x
    if (x == np.inf or x == np.NINF):
        return x
    #print("--------------------------------")
    signo,entero,decimal = convertirBinario(x)
    #print ("entero: ",entero," decimal: ",decimal)
    numPF = calzarExponente(signo,entero,decimal,bits_exp)
    #print ("numero pf: ",numPF)
    numPF = calzarMantisa(numPF,bits_mant)
    print ("numero pf: ",numPF)
    result = convertirDecimal(numPF,bits_mant,bits_exp,entero)
    result = float(result)
    print("numero final: ",result)
    return  result

"""
input
n1        : (double) valor a sumar
n2        : (double) valor a sumar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_suma(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la suma de dos numeros manteniendo el estandar asignado
    n1 = f_new_rep(n1,bits_mant,bits_exp)
    n2 = f_new_rep(n2,bits_mant,bits_exp)
    n3 = n1 + n2
    result = f_new_rep(n3,bits_mant,bits_exp)
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_multiplicacion(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la multiplicación de dos numeros manteniendo el estandar asignado
    n1 = f_new_rep(n1,bits_mant,bits_exp)
    n2 = f_new_rep(n2,bits_mant,bits_exp)
    n3 = n1 * n2
    result = f_new_rep(n3,bits_mant,bits_exp)
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_division(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la división de dos numeros manteniendo el estandar asignado
    n1 = f_new_rep(n1,bits_mant,bits_exp)
    n2 = f_new_rep(n2,bits_mant,bits_exp)
    if (n2 == 0):
        if (n1 == 0):
            return np.nan
        if (n1 > 0):
            return np.inf
        if (n1 < 0):
            return np.NINF
    n3 = n1 / n2
    result = f_new_rep(n3,bits_mant,bits_exp)
    return result

"""
input
A         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
x_roots      : (array) raices del polinomio ordenadas de menor a mayor
"""
def f_find_roots(A,bits_mant,bits_exp):
    # Algoritmo para encontrar raíces de f(x).
    x_roots = []
    ACuadrado = f_multiplicacion(A,A,bits_mant,bits_exp)
    #print("A^2: ",ACuadrado)
    ACuadradoMenosUno = f_suma(ACuadrado,-1.0,bits_mant,bits_exp)
    #print("A^2 - 1: ",ACuadradoMenosUno)
    raiz = f_new_rep(ACuadradoMenosUno**(0.5),bits_mant,bits_exp)
    #print("Raiz: ",raiz)
    
    #print("*****************************************************")
    
    #x1
    raizPartidoA = f_division(raiz,A,bits_mant,bits_exp)
    x1 = f_suma(1.0,raizPartidoA,bits_mant,bits_exp)
    
    #print("*****************************************************")
    
    #x2
    APorRaiz = f_multiplicacion(A,raiz,bits_mant,bits_exp)
    AcuadradoMasAPorRaiz = f_suma(ACuadrado,APorRaiz,bits_mant,bits_exp)
    x2 = f_division(1.0,AcuadradoMasAPorRaiz,bits_mant,bits_exp)
    
    #x3
    x3 = f_new_rep(1.0,bits_mant,bits_exp)
    
    x_roots.append(x1)
    x_roots.append(x2)
    x_roots.append(x3)
    
    x_roots.sort()
    
    return x_roots
#binarioEntero("10011001110101111001001101110010100101111000101110100")
f_new_rep(90812343322000000000000,52,11)
#f_find_roots(90812343322,52,11)

truncar 1
numero pf:  0 10001001011 0011001110101111001001101110010100101111000101110011
e 10011001110101111001001101110010100101111000101110011000000000000000000000000
d 
numero final:  9.0812343322e+22


9.0812343322e+22

### Sinusoidal de Oreman (40 puntos)
Considere la siguiente función en 3 variables:
\begin{equation}
T(t,\alpha,\beta) = \alpha\cdot \cos(t)+\beta\cdot \sin(\ln(t)).
\end{equation}

La cual permite calcular la temperatura de una cámara de ambiente dado los parámetros $\alpha$ y $\beta$ los cuales corresponden a las condiciones iniciales del experimento. 
Por otro lado $t$ representa el tiempo en días transcurridos desde que se inició.

A usted le permiten utilizar la cámara por un tiempo limitado, para esto planea terminar a los 25 días pero debe asegurarse que las sustancias sean extraídas a temperatura 0.

Cree un programa que reciba los parametros $\alpha$ y $\beta$ y que retorne el día en el cual deba ser extraído el material cumpliendo la condición de ser el día más cercano a 25 (ya sea antes o después).


## Explicación:
Se utilizó la función de bisección utilizada en clases, pero modificada.
Usando los parámetros a=0.1 y b=0.11, se fue sumando 0.01 a "b" hasta encontrar un cambio de signo. Luego se realiza la bisección normalmente.

<br>
Una vez que encuentra una raíz, "a" se actualiza hacia adelante para seguir buscando alguna raíz que se encuentre más cercana a 25 que nuestra raíz encontrada. Si no la encuentra o encuentra una más lejana, termina.

In [4]:
import bitstring as bs
import numpy as np

"""
input
alpha: (float) parametro alpha
beta: (float) parametro beta
tol: (float) tolerancia
output
root: (double) raiz obtenida
"""
def raiz_de_oreman(alpha, beta, tol = 1e-10):
    a = 0.1
    b = a + 0.01
    
    f = lambda x: alpha*np.cos(x)+ beta*np.sin(np.log(x))
    
    flag = True
    raiz = np.inf
    while (flag):
        fa = f(a)
        fb = f(b)
        if np.sign(f(a)*f(b)) >= 0:
            if (abs(25-raiz) < abs(25-b)):
                flag = False
                break                
            b+=0.01
        else:
            while(b-a)/2 > tol:
                c = (a+b)/2.
                fc = f(c)
                if fc == 0:
                    #print('f(c)==0')
                    break
                elif np.sign(fa*fc) < 0:
                    b = c
                    fb = fc
                else:
                    a = c
                    fa = fc
            root = (a+b)/2.
            #print("raiz encontrada: ",root)
            if(abs(25-root) < abs(25-raiz)):
                raiz = root
                a = raiz + 0.01
                b = a + 0.01
            else:
                flag = False
                break
    return raiz

# Referencias

Utilicé el código de bisección visto en clases y sacado del jupyter del profesor "04_roots_of_1D_equations.ipynb".